# Example modeling process

### Imports

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import (get_training_observations_gold, 
                     get_training_labels_gold, 
                     get_protein_proportions,
                     official_data_split_gold,
                     drop_empty_columns)

import pandas as pd
import numpy as np

# Model specific imports
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

### Load training data and labels

In [3]:
x_train_raw_counts = get_training_observations_gold()
x_train = get_protein_proportions(x_train_raw_counts)
print(f"There are {x_train.shape[1]} features")
y_train = get_training_labels_gold()



Getting all training observations from 'metagenome_classification.db'...
There are 16306 features
Getting all training labels from 'metagenome_classification.db'...


In [4]:
y_train.head()

index,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,HABITAT
0,Environmental,Aquatic,Marine,Marine
1,Environmental,Aquatic,Freshwater,Freshwater Sediment
2,Environmental,Aquatic,Marine,Marine sediment
3,Environmental,Aquatic,Marine,Marine
4,Environmental,Aquatic,Non-marine Saline and Alkaline,Hot spring


### Data manipulation
Validation splits, dimensionality reduction, etc

In [5]:
# Split into train/validation if not CV
X_tr, X_val, Y_tr, Y_val = train_test_split(x_train, y_train, test_size=0.2) #, random_state=1)

# Dimensionality reduction?
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
svd.fit(X_tr)

new_x_train = svd.transform(X_tr)
new_x_val = svd.transform(X_val)


In [6]:
# convert string labels to numeric
labels3 = [
    'Non-marine Saline and Alkaline',
    'Marine',
    'Soil',
    'Freshwater',
    'Defined media',
    'Composting',
    'Digestive system',
    'Thermal springs',
    'Rhizoplane',
    'Unclassified',
    'Mixed alcohol bioreactor',
    'Deep subsurface',
    'Tetrachloroethylene and derivatives',
    'Volcanic',
    'Outdoor Air',
    'Thiocyanate',
    'Bacteria',
    'Nutrient removal',
    'Phylloplane',
    'Rhizosphere',
    'Activated Sludge',
    'Gastrointestinal tract',
    'Sediment',
    'Endosphere',
    'Red algae',
    'Anaerobic digestor',
    'Peat moss',
    'Anaerobic'
]
labels3_map = {}
for i in range(0,len(labels3)):
    label = labels3[i]
    labels3_map[label] = i

Y_tr['GOLD_3_int'] = Y_tr['ECOSYSTEM_TYPE'].map(labels3_map)
Y_tr.head()
Y_val['GOLD_3_int'] = Y_val['ECOSYSTEM_TYPE'].map(labels3_map)

<ipython-input-6-f59e411cb5bc>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_tr['GOLD_3_int'] = Y_tr['ECOSYSTEM_TYPE'].map(labels3_map)
<ipython-input-6-f59e411cb5bc>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_val['GOLD_3_int'] = Y_val['ECOSYSTEM_TYPE'].map(labels3_map)


### Model training

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

def build_model(n_classes,
                hidden_layer_sizes=[],
                activation='relu',
                final_layer_activation='softmax',
                dropout=0.0,
                optimizer='Adam',
                learning_rate=0.01):
  """Build a multi-class logistic regression model using Keras.

  Args:
    n_classes: Number of output classes in the dataset.
    hidden_layer_sizes: A list with the number of units in each hidden layer.
    activation: The activation function to use for the hidden layers.
    optimizer: The optimizer to use (SGD, Adam).
    learning_rate: The desired learning rate for the optimizer.

  Returns:
    model: A tf.keras model (graph).
  """
  tf.keras.backend.clear_session()
  np.random.seed(0)
  tf.random.set_seed(0)
  model = keras.Sequential()
  model.add(keras.layers.Flatten())

  for hidden_layer_size in hidden_layer_sizes:
    model.add(keras.layers.Dense(hidden_layer_size))
    model.add(keras.layers.Activation(activation))
    if dropout > 0:
      model.add(keras.layers.Dropout(dropout))

  model.add(keras.layers.Dense(n_classes))
  model.add(keras.layers.Activation(final_layer_activation))
  opt = None
  if optimizer == 'SGD':
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
  elif optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  else:
    raise f"Unsupported optimizer, {optimizer}"
  model.compile(loss='sparse_categorical_crossentropy', 
    optimizer=opt, metrics=['accuracy'])    
  return model

def train_model(X_train, Y_train, num_classes,
                       hidden_layer_sizes=[],
                       activation='tanh',
                       final_layer_activation='softmax',
                       dropout=0.2,
                       optimizer='Adam',
                       learning_rate=0.01,
                       batch_size=64,
                       num_epochs=5):

  # Build the model.
  model = build_model(num_classes,
                      hidden_layer_sizes=hidden_layer_sizes,
                      activation=activation,
                      final_layer_activation=final_layer_activation,
                      dropout=dropout,
                      optimizer=optimizer,
                      learning_rate=learning_rate)

  # Train the model.
  print('Training...')
  history = model.fit(
    x=X_train,
    y=Y_train,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_split=0.1,
    verbose=1)

  model.summary()
  return model

nn3 = train_model(X_tr, Y_tr['GOLD_3_int'], 28,
    hidden_layer_sizes=[256, 256],
    dropout=0.2,
    optimizer='Adam',
    learning_rate=0.01,
    batch_size=128,
    num_epochs=100)


Training...
Epoch 1/100
8/8 [==============================] - 1s 47ms/step - loss: 2.3876 - accuracy: 0.2514 - val_loss: 2.0698 - val_accuracy: 0.1800
Epoch 2/100
8/8 [==============================] - 0s 31ms/step - loss: 1.7780 - accuracy: 0.3137 - val_loss: 1.6409 - val_accuracy: 0.5200
Epoch 3/100
8/8 [==============================] - 0s 32ms/step - loss: 1.3781 - accuracy: 0.5862 - val_loss: 1.3352 - val_accuracy: 0.4700
Epoch 4/100
8/8 [==============================] - 0s 32ms/step - loss: 1.1671 - accuracy: 0.5884 - val_loss: 1.1716 - val_accuracy: 0.6600
Epoch 5/100
8/8 [==============================] - 0s 31ms/step - loss: 0.9505 - accuracy: 0.7175 - val_loss: 0.9975 - val_accuracy: 0.7200
Epoch 6/100
8/8 [==============================] - 0s 31ms/step - loss: 0.8229 - accuracy: 0.7731 - val_loss: 1.1345 - val_accuracy: 0.6900
Epoch 7/100
8/8 [==============================] - 0s 32ms/step - loss: 0.7778 - accuracy: 0.7853 - val_loss: 0.8495 - val_accuracy: 0.7700
Epoch 8/

### Model evaluation

In [ ]:
# Scoring model

nn3_accuracy = nn3.evaluate(x=X_val, y=Y_val['GOLD_3_int'], verbose=0, return_dict=True)['accuracy']
print(f"Accuracy on GOLD 3: {nn3_accuracy}")